### BUILD, RUN AND PLOT A NEURAL NETWORK


In [20]:
%cd '/Users/juliencatanese/Dropbox/Datascientest/Projet_datascientest/Data_Kaggle/'
%pwd

import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical 

from time import time
from joblib import dump, load

import matplotlib.pyplot as plt
from matplotlib import cm
%matplotlib inline

from sklearn import metrics
from sklearn.metrics import classification_report

import itertools
import tensorflow as tf

# import tensorflow as tf

# Lecture des donnees PTB.csv
ptb_abnormal = pd.read_csv('./ptbdb_abnormal.csv', header=None)
ptb_normal = pd.read_csv('./ptbdb_normal.csv', header=None)

# Concatenation des deux DataFrames
ptb = pd.concat([ptb_normal, ptb_abnormal], ignore_index=True, sort=False)
ptb.rename(columns={187 :'class'}, inplace=True)

# Lecture des donnees MIT.csv
mit_train = pd.read_csv('./mitbih_train.csv', header=None)
mit_test = pd.read_csv('./mitbih_test.csv', header=None)

# Concatenation des deux DataFrames
mit = pd.concat([mit_train, mit_test], ignore_index=True, sort=False)
mit.rename(columns={187 :'class'}, inplace=True)

# Groupement des classes non-normal (1,2,3,4) en une seule classe Anormal (label=1)
mit['class'].replace({2:1,3:1,4:1}, inplace=True)

# Concatenation des jeux de donnees PTB et MIT  
data = pd.concat([ptb, mit], ignore_index=True, sort=False)

y = data['class'].copy()
X = data.drop('class', axis=1).copy()

X_train90, X_valid, y_train90, y_valid = train_test_split(X, 
                                                          y, 
                                                          train_size=0.9, 
                                                          random_state=1,
                                                          shuffle=True)
X_train, X_test, y_train, y_test = train_test_split(X_train90, 
                                                    y_train90, 
                                                    train_size=0.8, 
                                                    random_state=1,
                                                    shuffle=True)

print('X_train.shape=',X_train.shape)
print('X_test.shape=',X_test.shape)
print('X_valid.shape=',X_valid.shape)

print('y_train.shape=',y_train.shape)
y_train_cat = to_categorical(y_train, dtype='int')
print('y_train_cat.shape=',y_train_cat.shape)


inputs = tf.keras.Input(
    shape=X_train.shape[1],
    batch_size=None,
    name='inputs',
    dtype=None,
    sparse=None,
    tensor=None,
    ragged=None,
    type_spec=None)

ACTIVATION = 'relu'
n_Layers_liste= [1, 2, 4, 6] 
n_neurones= 300 

for n_Layers in n_Layers_liste:
    if n_Layers>=1:
        dense_layer1 = tf.keras.layers.Dense(units = int(n_neurones/n_Layers),
        name='dense_layer1_' + ACTIVATION, activation=ACTIVATION)
    elif n_Layers>=2:
        layer2 = True
        dense_layer2 = tf.keras.layers.Dense(units = int(n_neurones/n_Layers),
        name='dense_layer2_' + ACTIVATION, activation=ACTIVATION)
    elif n_Layers>=3:
        layer3 = True
        dense_layer3 = tf.keras.layers.Dense(units = int(n_neurones/n_Layers),
        name='dense_layer3_' + ACTIVATION, activation=ACTIVATION)
    elif n_Layers>=4:
        layer4 = True
        dense_layer4 = tf.keras.layers.Dense(units = int(n_neurones/n_Layers),
        name='dense_layer4_' + ACTIVATION, activation=ACTIVATION)
    elif n_Layers>=5:
        layer5 = True
        dense_layer5 = tf.keras.layers.Dense(units = int(n_neurones/n_Layers),
        name='dense_layer5_' + ACTIVATION, activation=ACTIVATION)
    elif n_Layers>=6:
        layer2 = True
        dense_layer6 = tf.keras.layers.Dense(units = int(n_neurones/n_Layers),
        name='dense_layer6_' + ACTIVATION, activation=ACTIVATION)

 
    dense_output = tf.keras.layers.Dense(
        units = 2,
        name='dense_output_softmax',
        activation='softmax',
        use_bias=True,
        kernel_initializer="normal",
        bias_initializer="zeros")
                                     
    FileName='L1';                                                                            
    x=dense_layer1(inputs)
    if layer2==True: 
        x=dense_layer2(x)
        FileName = 'L2'
        if layer3==True: 
            x=dense_layer3(x)
            FileName = 'L3'
            if layer4==True: 
                x=dense_layer4(x)
                FileName = 'L4'
                if layer5==True: 
                    x=dense_layer5(x)
                    FileName = 'L5'
                    if layer6==True: 
                        x=dense_layer6(x)
                        FileName = 'L6'
    outputs = dense_output(x)

    model = Model(inputs = inputs, outputs =outputs)
    model.summary()

    model.compile(loss= 'BinaryCrossentropy', optimizer= 'adam', metrics=['accuracy'])

    epoch = 150*10 ; print('epoch=', epoch)
    batch_size = 10000; print('batch_size=',batch_size)

    t0 = time()
    train_history = model.fit(X_train, 
                              y_train_cat, 
                              batch_size=batch_size, 
                              epochs=epoch, 
                              validation_split =0.2, 
                              callbacks = tf.keras.callbacks.EarlyStopping(patience=20))
    t1 = time() - t0
    print("===== Réalisé en {} secondes ======".format(round(t1,3)))

    
    SaveFileName = 'ANN_MaxLayers_' + FileName + '_TotalUnits' + str(n_neurones) + '.joblib'

    dump([model, params, train_history],SaveFileName)
    print('batch_size=',batch_size)
    print('Model saved as ', SaveFileName)

    print(model.summary())
    y_pred_cat = model.predict(X_test)
    y_pred = y_pred_cat.argmax(axis=1)
    print(classification_report(y_test, y_pred))


    train_set =train_history.history['accuracy']
    val_set =train_history.history['val_accuracy']

#     plt.figure(n_Layers)
#     plt.plot(train_set,label = 'train set');
#     plt.plot(val_set, label= 'validation set');
#     plt.ylabel('accuracy')
#     plt.xlabel('epochs    \n(with batch_size ='+  str(params['batch_size'])  +')')
#     plt.legend();
#     plt.ylim([0.85,1])
#     L1U= params['Layer1']['units']
#     L1A= params['Layer1']['activation']
#     TT = params['train_time']
#     plt.title(str(L1U)+'units '+L1A +' train time='+str(int(TT))+'s')
    


/Users/juliencatanese/Dropbox/Datascientest/Projet_datascientest/Data_Kaggle
X_train.shape= (89278, 187)
X_test.shape= (22320, 187)
X_valid.shape= (12400, 187)
y_train.shape= (89278,)
y_train_cat.shape= (89278, 2)
Model: "model_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 187)]             0         
                                                                 
 dense_layer1_relu (Dense)   (None, 300)               56400     
                                                                 
 dense_output_softmax (Dense  (None, 2)                602       
 )                                                               
                                                                 
Total params: 57,002
Trainable params: 57,002
Non-trainable params: 0
_________________________________________________________________
epoch= 1500
batch_size= 10000
Epoch 1/

Epoch 51/1500
8/8 [==============================] - 0s 31ms/step - loss: 0.2695 - accuracy: 0.8967 - val_loss: 0.2740 - val_accuracy: 0.8929
Epoch 52/1500
8/8 [==============================] - 0s 29ms/step - loss: 0.2678 - accuracy: 0.8978 - val_loss: 0.2731 - val_accuracy: 0.8937
Epoch 53/1500
8/8 [==============================] - 0s 28ms/step - loss: 0.2662 - accuracy: 0.8986 - val_loss: 0.2710 - val_accuracy: 0.8940
Epoch 54/1500
8/8 [==============================] - 0s 29ms/step - loss: 0.2644 - accuracy: 0.8992 - val_loss: 0.2691 - val_accuracy: 0.8963
Epoch 55/1500
8/8 [==============================] - 0s 29ms/step - loss: 0.2626 - accuracy: 0.8999 - val_loss: 0.2682 - val_accuracy: 0.8951
Epoch 56/1500
8/8 [==============================] - 0s 28ms/step - loss: 0.2609 - accuracy: 0.9004 - val_loss: 0.2665 - val_accuracy: 0.8953
Epoch 57/1500
8/8 [==============================] - 0s 28ms/step - loss: 0.2601 - accuracy: 0.8996 - val_loss: 0.2644 - val_accuracy: 0.8966
Epoch 

8/8 [==============================] - 0s 29ms/step - loss: 0.2022 - accuracy: 0.9219 - val_loss: 0.2114 - val_accuracy: 0.9172
Epoch 109/1500
8/8 [==============================] - 0s 29ms/step - loss: 0.2009 - accuracy: 0.9231 - val_loss: 0.2110 - val_accuracy: 0.9172
Epoch 110/1500
8/8 [==============================] - 0s 28ms/step - loss: 0.2003 - accuracy: 0.9232 - val_loss: 0.2120 - val_accuracy: 0.9142
Epoch 111/1500
8/8 [==============================] - 0s 29ms/step - loss: 0.1997 - accuracy: 0.9228 - val_loss: 0.2095 - val_accuracy: 0.9177
Epoch 112/1500
8/8 [==============================] - 0s 28ms/step - loss: 0.1985 - accuracy: 0.9233 - val_loss: 0.2093 - val_accuracy: 0.9163
Epoch 113/1500
8/8 [==============================] - 0s 28ms/step - loss: 0.1979 - accuracy: 0.9237 - val_loss: 0.2083 - val_accuracy: 0.9188
Epoch 114/1500
8/8 [==============================] - 0s 28ms/step - loss: 0.1978 - accuracy: 0.9241 - val_loss: 0.2080 - val_accuracy: 0.9169
Epoch 115/1500

8/8 [==============================] - 0s 29ms/step - loss: 0.1647 - accuracy: 0.9385 - val_loss: 0.1801 - val_accuracy: 0.9292
Epoch 166/1500
8/8 [==============================] - 0s 28ms/step - loss: 0.1649 - accuracy: 0.9383 - val_loss: 0.1791 - val_accuracy: 0.9302
Epoch 167/1500
8/8 [==============================] - 0s 28ms/step - loss: 0.1641 - accuracy: 0.9390 - val_loss: 0.1780 - val_accuracy: 0.9313
Epoch 168/1500
8/8 [==============================] - 0s 28ms/step - loss: 0.1635 - accuracy: 0.9393 - val_loss: 0.1783 - val_accuracy: 0.9297
Epoch 169/1500
8/8 [==============================] - 0s 28ms/step - loss: 0.1627 - accuracy: 0.9394 - val_loss: 0.1776 - val_accuracy: 0.9308
Epoch 170/1500
8/8 [==============================] - 0s 28ms/step - loss: 0.1622 - accuracy: 0.9401 - val_loss: 0.1775 - val_accuracy: 0.9302
Epoch 171/1500
8/8 [==============================] - 0s 28ms/step - loss: 0.1614 - accuracy: 0.9400 - val_loss: 0.1764 - val_accuracy: 0.9326
Epoch 172/1500

8/8 [==============================] - 0s 37ms/step - loss: 0.1386 - accuracy: 0.9503 - val_loss: 0.1581 - val_accuracy: 0.9420
Epoch 223/1500
8/8 [==============================] - 0s 42ms/step - loss: 0.1395 - accuracy: 0.9502 - val_loss: 0.1619 - val_accuracy: 0.9359
Epoch 224/1500
8/8 [==============================] - 0s 37ms/step - loss: 0.1386 - accuracy: 0.9503 - val_loss: 0.1573 - val_accuracy: 0.9415
Epoch 225/1500
8/8 [==============================] - 0s 37ms/step - loss: 0.1379 - accuracy: 0.9505 - val_loss: 0.1571 - val_accuracy: 0.9404
Epoch 226/1500
8/8 [==============================] - 0s 36ms/step - loss: 0.1375 - accuracy: 0.9511 - val_loss: 0.1566 - val_accuracy: 0.9389
Epoch 227/1500
8/8 [==============================] - 0s 38ms/step - loss: 0.1367 - accuracy: 0.9504 - val_loss: 0.1559 - val_accuracy: 0.9412
Epoch 228/1500
8/8 [==============================] - 0s 37ms/step - loss: 0.1369 - accuracy: 0.9509 - val_loss: 0.1581 - val_accuracy: 0.9379
Epoch 229/1500

...

8/8 [==============================] - 0s 21ms/step - loss: 0.1661 - accuracy: 0.9378 - val_loss: 0.1882 - val_accuracy: 0.9263
Epoch 681/1500
8/8 [==============================] - 0s 20ms/step - loss: 0.1660 - accuracy: 0.9377 - val_loss: 0.1883 - val_accuracy: 0.9264
Epoch 682/1500
8/8 [==============================] - 0s 20ms/step - loss: 0.1658 - accuracy: 0.9380 - val_loss: 0.1882 - val_accuracy: 0.9269
Epoch 683/1500
8/8 [==============================] - 0s 20ms/step - loss: 0.1658 - accuracy: 0.9378 - val_loss: 0.1882 - val_accuracy: 0.9262
Epoch 684/1500
8/8 [==============================] - 0s 20ms/step - loss: 0.1657 - accuracy: 0.9377 - val_loss: 0.1881 - val_accuracy: 0.9275
Epoch 685/1500
8/8 [==============================] - 0s 20ms/step - loss: 0.1661 - accuracy: 0.9380 - val_loss: 0.1884 - val_accuracy: 0.9257
Epoch 686/1500
8/8 [==============================] - 0s 20ms/step - loss: 0.1660 - accuracy: 0.9375 - val_loss: 0.1885 - val_accuracy: 0.9261
Epoch 687/1500

8/8 [==============================] - 0s 21ms/step - loss: 0.1621 - accuracy: 0.9392 - val_loss: 0.1855 - val_accuracy: 0.9275
Epoch 738/1500
8/8 [==============================] - 0s 20ms/step - loss: 0.1620 - accuracy: 0.9395 - val_loss: 0.1854 - val_accuracy: 0.9280
Epoch 739/1500
8/8 [==============================] - 0s 20ms/step - loss: 0.1619 - accuracy: 0.9390 - val_loss: 0.1852 - val_accuracy: 0.9283
Epoch 740/1500
8/8 [==============================] - 0s 20ms/step - loss: 0.1620 - accuracy: 0.9390 - val_loss: 0.1872 - val_accuracy: 0.9270
Epoch 741/1500
8/8 [==============================] - 0s 20ms/step - loss: 0.1622 - accuracy: 0.9391 - val_loss: 0.1851 - val_accuracy: 0.9281
Epoch 742/1500
8/8 [==============================] - 0s 20ms/step - loss: 0.1619 - accuracy: 0.9393 - val_loss: 0.1858 - val_accuracy: 0.9281
Epoch 743/1500
8/8 [==============================] - 0s 20ms/step - loss: 0.1618 - accuracy: 0.9389 - val_loss: 0.1853 - val_accuracy: 0.9278
Epoch 744/1500

8/8 [==============================] - 0s 20ms/step - loss: 0.1588 - accuracy: 0.9406 - val_loss: 0.1827 - val_accuracy: 0.9286
Epoch 795/1500
8/8 [==============================] - 0s 21ms/step - loss: 0.1583 - accuracy: 0.9406 - val_loss: 0.1827 - val_accuracy: 0.9294
Epoch 796/1500
8/8 [==============================] - 0s 21ms/step - loss: 0.1583 - accuracy: 0.9406 - val_loss: 0.1831 - val_accuracy: 0.9289
Epoch 797/1500
8/8 [==============================] - 0s 22ms/step - loss: 0.1594 - accuracy: 0.9397 - val_loss: 0.1826 - val_accuracy: 0.9291
Epoch 798/1500
8/8 [==============================] - 0s 20ms/step - loss: 0.1592 - accuracy: 0.9404 - val_loss: 0.1849 - val_accuracy: 0.9274
Epoch 799/1500
8/8 [==============================] - 0s 21ms/step - loss: 0.1588 - accuracy: 0.9404 - val_loss: 0.1833 - val_accuracy: 0.9286
Epoch 800/1500
8/8 [==============================] - 0s 21ms/step - loss: 0.1580 - accuracy: 0.9407 - val_loss: 0.1826 - val_accuracy: 0.9288
Epoch 801/1500

8/8 [==============================] - 0s 22ms/step - loss: 0.1549 - accuracy: 0.9418 - val_loss: 0.1809 - val_accuracy: 0.9293
Epoch 852/1500
8/8 [==============================] - 0s 21ms/step - loss: 0.1551 - accuracy: 0.9416 - val_loss: 0.1810 - val_accuracy: 0.9310
Epoch 853/1500
8/8 [==============================] - 0s 21ms/step - loss: 0.1557 - accuracy: 0.9418 - val_loss: 0.1817 - val_accuracy: 0.9290
Epoch 854/1500
8/8 [==============================] - 0s 21ms/step - loss: 0.1561 - accuracy: 0.9416 - val_loss: 0.1856 - val_accuracy: 0.9278
Epoch 855/1500
8/8 [==============================] - 0s 21ms/step - loss: 0.1560 - accuracy: 0.9414 - val_loss: 0.1812 - val_accuracy: 0.9294
Epoch 856/1500
8/8 [==============================] - 0s 21ms/step - loss: 0.1549 - accuracy: 0.9418 - val_loss: 0.1808 - val_accuracy: 0.9296
Epoch 857/1500
8/8 [==============================] - 0s 21ms/step - loss: 0.1546 - accuracy: 0.9419 - val_loss: 0.1804 - val_accuracy: 0.9299
Epoch 858/1500

8/8 [==============================] - 0s 20ms/step - loss: 0.1518 - accuracy: 0.9433 - val_loss: 0.1805 - val_accuracy: 0.9296
Epoch 909/1500
8/8 [==============================] - 0s 20ms/step - loss: 0.1522 - accuracy: 0.9430 - val_loss: 0.1827 - val_accuracy: 0.9284
Epoch 910/1500
8/8 [==============================] - 0s 20ms/step - loss: 0.1525 - accuracy: 0.9431 - val_loss: 0.1785 - val_accuracy: 0.9320
Epoch 911/1500
8/8 [==============================] - 0s 20ms/step - loss: 0.1523 - accuracy: 0.9431 - val_loss: 0.1788 - val_accuracy: 0.9314
Epoch 912/1500
8/8 [==============================] - 0s 21ms/step - loss: 0.1520 - accuracy: 0.9430 - val_loss: 0.1796 - val_accuracy: 0.9302
Epoch 913/1500
8/8 [==============================] - 0s 20ms/step - loss: 0.1518 - accuracy: 0.9433 - val_loss: 0.1788 - val_accuracy: 0.9316
Epoch 914/1500
8/8 [==============================] - 0s 20ms/step - loss: 0.1517 - accuracy: 0.9433 - val_loss: 0.1788 - val_accuracy: 0.9315
Epoch 915/1500